In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import winsound
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

In [2]:
raw_df = pd.read_csv('data\M1_2019.csv')
df = raw_df[['TS', 'Close']]
#df = df[:10000]
df.head()

,TS,Close
0,2019-01-01 17:02,1.14598
1,2019-01-01 17:03,1.14607
2,2019-01-01 17:04,1.14606
3,2019-01-01 17:05,1.14621
4,2019-01-01 17:06,1.14665


In [3]:
def new_df(roll_len):
    col_list = []

    for i in range(roll_len):
        col_list.append(i)
    
    col_list.append('TS')
    return(col_list)

In [4]:
def reorder_df(df, roll_len):
    tot_list = []
    for i, val in tqdm(df.iterrows()):
        act_row = i+roll_len-1
        l = list(df['Close'][i:act_row+1])    
        if len(l) == roll_len:
            tot_list.append(l)
            l.append(df['TS'][act_row])
    df1=pd.DataFrame(tot_list,columns=col_list)
    return(df1)

In [5]:
def get_year(row):
    year_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').date().year
    return(year_val)

def get_month(row):
    month_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').date().month
    return(month_val)

def get_day(row):
    day_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').date().day
    return(day_val)   

def get_hour(row):
    hour_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').time().hour
    return(hour_val)   

def get_min(row):
    min_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').time().minute
    return(min_val)   

def get_dir(row):
    diff = row[roll_len-1] - row[roll_len-2]
    
    if diff > 0.0002:
        direction = 'increase'
    elif diff < -0.0002:
        direction = 'decrease'
    else:
        direction = 'same'        
    
    return(direction)

def split_ts(df1):
    #df1['year'] = df1.apply(get_year, axis=1)
    df1['month'] = df1.apply(get_month, axis=1)
    df1['day'] = df1.apply(get_day, axis=1)
    df1['hour'] = df1.apply(get_hour, axis=1)
    df1['min'] = df1.apply(get_min, axis=1)
    df1['direction'] = df1.apply(get_dir, axis=1)
    
    cols = list(df1)

    cols.insert(0, cols.pop(cols.index('min')))
    cols.insert(0, cols.pop(cols.index('hour')))
    cols.insert(0, cols.pop(cols.index('day')))
    cols.insert(0, cols.pop(cols.index('month')))
    #cols.insert(0, cols.pop(cols.index('year')))

    df1 = df1[cols]

    del df1['TS']
    #del df1[roll_len-1]
    return(df1)

In [6]:
roll_len = 15
col_list = new_df(roll_len)
rolled_df = reorder_df(df, roll_len)

372530it [01:06, 5594.51it/s]


In [7]:
%%time
cols = list(rolled_df)
cols.insert(0, cols.pop(cols.index('TS')))
rolled_df = rolled_df[cols]

Wall time: 29.9 ms


In [8]:
%%time 
rolled_df = split_ts(rolled_df)

Wall time: 48.7 s


In [9]:
%%time
rolled_df.to_csv('data/rolled_df.csv', index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Wall time: 4.73 s


In [10]:
rolled_df.head()

,month,day,hour,min,0,1,2,3,4,5,6,7,8,9,10,11,12,13,direction
0,1,1,17,16,1.14598,1.14607,1.14606,1.14621,1.14665,1.14607,1.14607,1.14606,1.14607,1.14607,1.14608,1.14607,1.14608,1.14607,same
1,1,1,17,17,1.14607,1.14606,1.14621,1.14665,1.14607,1.14607,1.14606,1.14607,1.14607,1.14608,1.14607,1.14608,1.14607,1.14607,same
2,1,1,17,18,1.14606,1.14621,1.14665,1.14607,1.14607,1.14606,1.14607,1.14607,1.14608,1.14607,1.14608,1.14607,1.14607,1.14607,same
3,1,1,17,19,1.14621,1.14665,1.14607,1.14607,1.14606,1.14607,1.14607,1.14608,1.14607,1.14608,1.14607,1.14607,1.14607,1.14607,same
4,1,1,17,20,1.14665,1.14607,1.14607,1.14606,1.14607,1.14607,1.14608,1.14607,1.14608,1.14607,1.14607,1.14607,1.14607,1.14606,same


In [12]:
rolled_df['direction'].value_counts()

same        355148
decrease      8977
increase      8391
Name: direction, dtype: int64